# Agent Memory

Add conversational memory using checkpointers and persistent storage.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
system_prompt = "You are a helpful assistant."

## Problem: Agent Without Memory

In [ ]:
# Agent without memory loses context
agent_no_memory = create_agent(model=model, system_prompt=system_prompt)

# First question
r1 = agent_no_memory.invoke({'messages': [HumanMessage("My name is John")]})
r1['messages'][-1].text

In [ ]:
# Follow-up - agent doesn't remember
r2 = agent_no_memory.invoke({'messages': [HumanMessage("What's my name?")]})
r2['messages'][-1].text

## Solution: SQLite Checkpointer

In [ ]:
# Create checkpointer for persistent memory
conn = sqlite3.connect("data/agent_memory.db", check_same_thread=False)
checkpointer = SqliteSaver(conn=conn)

agent_memory = create_agent(
    model=model,
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

## Using Thread IDs for Sessions

In [ ]:
# Each thread_id maintains separate conversation
config = {"configurable": {"thread_id": "user_123"}}

# First message
r1 = agent_memory.invoke(
    {'messages': [HumanMessage("My name is John")]},
    config=config
)
r1['messages'][-1].text

In [ ]:
# Follow-up - agent remembers!
r2 = agent_memory.invoke(
    {'messages': [HumanMessage("What's my name?")]},
    config=config
)
r2['messages'][-1].text

In [ ]:
# Continue conversation
r3 = agent_memory.invoke(
    {'messages': [HumanMessage("What did I tell you earlier?")]},
    config=config
)
r3['messages'][-1].text

## Multiple Conversation Sessions

In [ ]:
# Different thread IDs = separate contexts
session_a = {"configurable": {"thread_id": "session_a"}}
session_b = {"configurable": {"thread_id": "session_b"}}

# Session A
agent_memory.invoke(
    {'messages': [HumanMessage("I'm interested in Python")]},
    session_a
)

# Session B
agent_memory.invoke(
    {'messages': [HumanMessage("I'm interested in JavaScript")]},
    session_b
)

In [ ]:
# Each session maintains its own context
r_a = agent_memory.invoke(
    {'messages': [HumanMessage("What am I interested in?")]},
    session_a
)
print("Session A:", r_a['messages'][-1].text)

r_b = agent_memory.invoke(
    {'messages': [HumanMessage("What am I interested in?")]},
    session_b
)
print("Session B:", r_b['messages'][-1].text)

## Inspecting Conversation History

In [ ]:
# View message count and types
response = agent_memory.invoke(
    {'messages': [HumanMessage("Tell me about Python")]},
    session_a
)

print(f"Total messages: {len(response['messages'])}")

from langchain.messages import HumanMessage as HM, AIMessage as AM
human_count = sum(1 for m in response['messages'] if isinstance(m, HM))
ai_count = sum(1 for m in response['messages'] if isinstance(m, AM))

print(f"Human: {human_count}, AI: {ai_count}")

## Key Takeaways

- Checkpointers persist conversation state
- Thread IDs manage separate sessions
- SQLite provides lightweight persistence
- Memory enables multi-turn conversations
- History grows with each interaction

## Exercise

Create a multi-turn conversation and experiment with different thread IDs.

In [ ]:
# Your code here
